# Robyn: Marketing Mix Modeling Application

This notebook demonstrates the usage of Robyn, a Marketing Mix Modeling (MMM) application. We'll go through the main steps of initializing the model, running it, and performing budget allocation.



## 1. Import Required Libraries and Create Synthetic Data

First, let's import the necessary libraries and create some synthetic data for our demonstration.

In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from robyn.robyn import Robyn
from robyn.data.entities.mmmdata import MMMData
from robyn.data.entities.holidays_data import HolidaysData
from robyn.data.entities.hyperparameters import Hyperparameters, ChannelHyperparameters
from robyn.data.entities.calibration_input import CalibrationInput, ChannelCalibrationData
from robyn.data.entities.enums import AdstockType, DependentVarType, CalibrationScope
from robyn.modeling.entities.modelrun_trials_config import TrialsConfig
from robyn.analysis.budget_allocator import BudgetAllocatorConfig


## 2.1 Create sythetic Data

In [ ]:
# Create synthetic data
def create_synthetic_data(start_date, end_date):
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    n = len(date_range)
    
    data = pd.DataFrame({
        'date': date_range,
        'revenue': np.random.normal(1000, 100, n),
        'tv_spend': np.random.normal(500, 50, n),
        'radio_spend': np.random.normal(300, 30, n),
        'ooh_spend': np.random.normal(200, 20, n),
        'print_spend': np.random.normal(100, 10, n),
        'competitor_sales': np.random.normal(800, 80, n),
        'temperature': np.random.normal(20, 5, n)
    })
    
    return data

# Generate synthetic data
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)
synthetic_data = create_synthetic_data(start_date, end_date)

# Display the first few rows of the synthetic data
print(synthetic_data.head())

## 2.2. Initialize Robyn

Now, let's initialize Robyn with our synthetic data and configuration.

In [ ]:
# Initialize Robyn
robyn = Robyn(working_dir="~/temp/robyn")

# Create MMMData
mmm_data_spec = MMMData.MMMDataSpec(
    dep_var="revenue",
    dep_var_type=DependentVarType.REVENUE,
    date_var="date",
    paid_media_spends=["tv_spend", "radio_spend", "ooh_spend", "print_spend"],
    paid_media_vars=["tv_spend", "radio_spend", "ooh_spend", "print_spend"],
    context_vars=["competitor_sales", "temperature"],
)

mmm_data = MMMData(synthetic_data, mmm_data_spec)

# Create HolidaysData (using dummy data for demonstration)
holidays_data = HolidaysData(
    dt_holidays=pd.DataFrame(),
    prophet_vars=[],
    prophet_signs=[],
    prophet_country="US",
    day_interval=1
)

# Create Hyperparameters
hyperparameters = Hyperparameters({
    "tv_spend": ChannelHyperparameters(
        thetas=[0.1, 0.9],
        alphas=[0.5, 3.0],
        gammas=[0.3, 1.0],
        shapes=[0.0001, 2.0],
        scales=[0.0001, 2.0],
    ),
    "radio_spend": ChannelHyperparameters(
        thetas=[0.1, 0.9],
        alphas=[0.5, 3.0],
        gammas=[0.3, 1.0],
        shapes=[0.0001, 2.0],
        scales=[0.0001, 2.0],
    ),
    "ooh_spend": ChannelHyperparameters(
        thetas=[0.1, 0.9],
        alphas=[0.5, 3.0],
        gammas=[0.3, 1.0],
        shapes=[0.0001, 2.0],
        scales=[0.0001, 2.0],
    ),
    "print_spend": ChannelHyperparameters(
        thetas=[0.1, 0.9],
        alphas=[0.5, 3.0],
        gammas=[0.3, 1.0],
        shapes=[0.0001, 2.0],
        scales=[0.0001, 2.0],
    ),
})

# Create CalibrationInput (using dummy data for demonstration)
calibration_input = CalibrationInput({
    "tv_spend": ChannelCalibrationData(
        lift_start_date=pd.Timestamp("2022-03-01"),
        lift_end_date=pd.Timestamp("2022-03-15"),
        lift_abs=10000,
        spend=50000,
        confidence=0.9,
        metric="revenue",
        calibration_scope=CalibrationScope.IMMEDIATE
    )
})

# Initialize Robyn
robyn.initialize(
    mmm_data=mmm_data,
    holidays_data=holidays_data,
    hyperparameters=hyperparameters,
    calibration_input=calibration_input
)

print("Robyn initialized successfully!")

## 3. Run Robyn Model

After initialization, we can run the Robyn model.

In [ ]:
# Configure trials
trials_config = TrialsConfig(
    num_trials=5,
    num_iterations_per_trial=1000,
    timeseries_validation=True,
    add_penalty_factor=True
)

# Run the model
model_output = robyn.model_run(
    num_of_cores=4,
    trials_config=trials_config
)

print("Model run completed successfully!")
print(f"Number of trials: {len(model_output.trials)}")

## 4. Budget Allocation

Finally, let's perform budget allocation using the trained model.

In [ ]:
# Configure budget allocator
budget_allocator_config = BudgetAllocatorConfig(
    scenario="max_response",
    total_budget=1000000,
    target_value=0,
    date_range="2023-01-01,2023-12-31",
    channel_constr_low=0.7,
    channel_constr_up=1.3,
    channel_constr_multiplier=3,
    optim_algo="SLSQP_AUGLAG",
    maxeval=100000,
    constr_mode="eq"
)

# Run budget allocation
budget_allocation_result = robyn.budget_allocator(
    select_model="1",  # Assuming we're using the first model
    budger_allocator_config=budget_allocator_config
)

print("Budget allocation completed successfully!")
print(budget_allocation_result)


This notebook demonstrates the basic workflow of using Robyn for Marketing Mix Modeling. In a real-world scenario, you would need to replace the synthetic data with your actual marketing data and adjust the parameters accordingly.

Remember to explore the full capabilities of Robyn, including model evaluation, visualization, and interpretation of results, which are beyond the scope of this basic demonstration.